## Objective:
This is the second notebook in the assignment and deals with: 
- Reading data from the staging tables and querying them to derive necessary metrics
- Creating a curated database and some tables in it, and
- Storing the metrics in the relevant tables in the curated database.

The databricks public link to this workbook is https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/1721482899250574/754950429162319/2654077590604412/latest.html

### 1. Staging database

In [ ]:
spark.conf.set("spark.sql.legacy.timeParserPolicy","LEGACY")

In [ ]:
%sql
-- Run the following command to view all the tables present in the staging schema

USE staging;
SHOW TABLES;

database,tableName,isTemporary
staging,customers,false
staging,orders,false


### 1a) Reading data from the staging database


In [ ]:
from pyspark.sql.functions import *

orders_df = spark.read.table("staging.orders")
orders_df.cache()

customers_df = spark.read.table("staging.customers")
customers_df.cache()

Out[3]: DataFrame[City: string, County: string, CustomerSince: date, Email: string, Gender: string, PlaceName: string, Region: string, State: string, Zip: string, Age: int, CustID: int, FullName: string]

In [ ]:
#Run the following command to view the unique values present in the "Status" column in the orders_df dataframe.
orders_df.select('Status').distinct().collect()

Out[4]: [Row(Status='received'),
 Row(Status='cod'),
 Row(Status='holded'),
 Row(Status='canceled'),
 Row(Status='refund'),
 Row(Status='paid'),
 Row(Status='pending'),
 Row(Status='closed'),
 Row(Status='payment_review'),
 Row(Status='order_refunded'),
 Row(Status='processing'),
 Row(Status='complete'),
 Row(Status='pending_paypal')]

##### From the above output, we find that there are orders with statuses like "holded", "canceled", "pending_paypal" etc. that do not count towards final *revenue* to the company. And there are some statuses like "order_refunded', 'refund' that we need to consider separately as *refunds*.
- We therefore split the orders table into three parts - completed orders, refunded orders and all other orders - and calculate the aggregates.

In [ ]:
completed_statuses = ['received','cod','paid','closed','complete']
refunded_statuses = ['refund','order_refunded']

completed_orders =  orders_df.filter(orders_df.Status.isin(completed_statuses))
refunded_orders =   orders_df.filter(orders_df.Status.isin(refunded_statuses))
other_orders =      orders_df.filter(~orders_df.Status.isin(completed_statuses + refunded_statuses))

In [ ]:
#Run the following command to check if an order occurs more than once in the dataset.

from pyspark.sql import functions as F

orders_df.groupBy('OrderID').agg(F.count('*').alias('Count')).filter('Count > 1').show(10)

+---------+-----+
|  OrderID|Count|
+---------+-----+
|100433271|    2|
|100436108|    2|
|100437512|    2|
|100465160|    3|
|100438156|    2|
|100538093|    2|
|100445454|    2|
|100445617|    3|
|100536975|    2|
|100444899|    2|
+---------+-----+
only showing top 10 rows



##### As can be seen from the above output, there are multiple occurences of some orders. We, therefore, will consider only the unique occurences when calculating the *_total orders_* placed by a customer.

### 2. Querying the above dataframes to derive metrics to answer the questions

##### 2a) Create new dataframes by calculating the total revenue/refund and the number of orders placed, grouped by the product category, purchase year and month-year.
- This helps us answer questions 1,2,3 in the assignment.

In [ ]:
completed_orders_metrics = completed_orders.groupBy("Category", "Year", "Month_Year").agg( sum("Total").alias("Revenue"),countDistinct("OrderID").alias("TotalOrders"))\
                                                                                     .orderBy(['Year','Month_Year'],ascending=True
                                                    )

In [ ]:
refunded_orders_metrics = refunded_orders.groupBy("Category", "Year", "Month_Year").agg( sum("Total").alias("Refund"),countDistinct("OrderID").alias("TotalOrders")) \
                                                                                   .orderBy(['Year','Month_Year'],ascending=True
                                                                                   )

In [ ]:
other_orders_metrics = other_orders.groupBy("Category", "Year", "Month_Year").agg( sum("Total").alias("Revenue"),countDistinct("OrderID").alias("TotalOrders")) \
                                                                                   .orderBy(['Year','Month_Year'],ascending=True
                                                                                   )

In [ ]:
#Show the first 5 records
completed_orders_metrics.show(5)

+---------------+----+----------+------------------+-----------+
|       Category|Year|Month_Year|           Revenue|TotalOrders|
+---------------+----+----------+------------------+-----------+
|  Home & Living|2020|  Dec-2020|186017.76805210114|       1724|
|      Computing|2020|  Dec-2020|1252693.5991449356|       1173|
|Women's Fashion|2020|  Dec-2020| 598978.1214361191|       2623|
|     Appliances|2020|  Dec-2020| 5082387.872704029|       6989|
|    Kids & Baby|2020|  Dec-2020| 57219.67628479004|        640|
+---------------+----+----------+------------------+-----------+
only showing top 5 rows



##### 2b) Create a new dataframe by joining the *_completed_orders_* with the customers_df on the CustID column.
- This helps us answer questions 4,5 and 6 in the assignment.
- Please note that only **completed orders** have been considered for calculating the **spend** by customers.

In [ ]:
completed_customer_orders = completed_orders.join(customers_df, on = ["CustID"], how = 'inner' )

In [ ]:
completed_customer_orders.show(5)

+------+---------+----------+--------+------+----------------+-----+-----+---------------+-----+-------------+--------------+----------+----+--------+-------+-------------+--------------------+------+---------+---------+-----+-----+---+----------------+
|CustID|  OrderID|Order_Date|  Status|ItemID|Quantity_Ordered|Price|Value|Discount_Amount|Total|     Category|Payment_Method|Month_Year|Year|    City| County|CustomerSince|               Email|Gender|PlaceName|   Region|State|  Zip|Age|        FullName|
+------+---------+----------+--------+------+----------------+-----+-----+---------------+-----+-------------+--------------+----------+----+--------+-------+-------------+--------------------+------+---------+---------+-----+-----+---+----------------+
| 24389|100433019|2021-01-06|received|708346|               2| 29.8| 29.8|            0.0| 29.8|Men's Fashion|           cod|  Jan-2021|2021|Palmetto|Manatee|   2014-10-14|maximo.herring@ho...|     M| Palmetto|    South|   FL|34220| 45| H

In [ ]:
# Derive a new column "Customer Segment" based on the following logic
# If Age is between 0-20, then Young.
# If Age is between 20-35, then Adults. 
# If Age is between 35-55, then Middle Ages.
# If Age is greater than 55, then Old.


from pyspark.sql.functions import when

completed_customer_orders = completed_customer_orders.withColumn('Customer_Segment', when(completed_customer_orders.Age <= 20, "Young")
                                                   .when((completed_customer_orders.Age > 20) & (completed_customer_orders.Age <= 35), "Adults")
                                                   .when((completed_customer_orders.Age > 35) & (completed_customer_orders.Age <= 55), "Middle Ages")
                                                   .otherwise("Old")
                                                    )

completed_customer_orders.show(5)                                                                  

+------+---------+----------+--------+------+----------------+------+------+---------------+------+-----------------+--------------+----------+----+----------+--------+-------------+--------------------+------+----------+-------+-----+-----+---+------------------+----------------+
|CustID|  OrderID|Order_Date|  Status|ItemID|Quantity_Ordered| Price| Value|Discount_Amount| Total|         Category|Payment_Method|Month_Year|Year|      City|  County|CustomerSince|               Email|Gender| PlaceName| Region|State|  Zip|Age|          FullName|Customer_Segment|
+------+---------+----------+--------+------+----------------+------+------+---------------+------+-----------------+--------------+----------+----+----------+--------+-------------+--------------------+------+----------+-------+-----+-----+---+------------------+----------------+
| 82529|100433524|2021-01-08|complete|709387|               2| 109.9| 109.9|            0.0| 109.9|Beauty & Grooming|           cod|  Jan-2021|2021|     M

#### 2c) RFM calculation for each customer.
- *Recency* indicates the number of weeks that have elapsed between the customer's last purchase date and the reference date (31st Oct 2021)
- *Frequency* indicates the number of *_unique_* purchases made by a customer during a certain time period
- *Monetary* value indicates the total amount of money a customer has spent on purchases.

Even here only the **completed orders** have been considered for the calculations.


In [ ]:
# Calculate "Frequency" and "Monetary" fields along with the last purchase date

import datetime
from pyspark.sql.functions import lit,ceil, datediff

rfm_df = completed_customer_orders.groupBy(['CustID','FullName']).agg( max("Order_Date").alias("Last_Purchase_Date"), countDistinct("OrderID").alias("Frequency"),sum("Total").alias("Monetary")).orderBy("CustID",ascending=True)

rfm_df.show(5)

+------+----------------+------------------+---------+------------------+
|CustID|        FullName|Last_Purchase_Date|Frequency|          Monetary|
+------+----------------+------------------+---------+------------------+
|     4| Doughty, Reggie|        2021-09-29|       18| 21635.94997024536|
|    15| Diebold, Debbie|        2021-02-11|        3|216.80000114440918|
|    20|    Pulver, Eddy|        2021-09-29|        7| 23702.40003967285|
|    21|       Kan, Adam|        2021-02-03|        1|             105.0|
|    23|Bostwick, Roscoe|        2021-08-07|        2|393.23999786376953|
+------+----------------+------------------+---------+------------------+
only showing top 5 rows



In [ ]:
# Calculate "Recency" as the weeks difference between the last_purchase_date and 2021-10-31 (the reference date)

reference_date=datetime.date(2021,10,31)

rfm_df = rfm_df.withColumn("Reference_Date", lit(reference_date))

rfm_df = rfm_df.withColumn('Recency', ceil(datediff('Reference_Date','Last_Purchase_Date')/7))

rfm_df.show(5)


+------+----------------+------------------+---------+------------------+--------------+-------+
|CustID|        FullName|Last_Purchase_Date|Frequency|          Monetary|Reference_Date|Recency|
+------+----------------+------------------+---------+------------------+--------------+-------+
|     4| Doughty, Reggie|        2021-09-29|       18| 21635.94997024536|    2021-10-31|      5|
|    15| Diebold, Debbie|        2021-02-11|        3|216.80000114440918|    2021-10-31|     38|
|    20|    Pulver, Eddy|        2021-09-29|        7| 23702.40003967285|    2021-10-31|      5|
|    21|       Kan, Adam|        2021-02-03|        1|             105.0|    2021-10-31|     39|
|    23|Bostwick, Roscoe|        2021-08-07|        2|393.23999786376953|    2021-10-31|     13|
+------+----------------+------------------+---------+------------------+--------------+-------+
only showing top 5 rows



In [ ]:
#Finally, drop the columns Last_Purchase_Date and Reference_Date and re-arrange the columns

rfm_df = rfm_df.drop("Last_Purchase_Date", "Reference_Date")

# Re-arrange the columns
rfm_df = rfm_df.select("CustID", "FullName", "Recency", "Frequency", "Monetary")

# Show the resulting dataframe
rfm_df.show(5)

+------+----------------+-------+---------+------------------+
|CustID|        FullName|Recency|Frequency|          Monetary|
+------+----------------+-------+---------+------------------+
|     4| Doughty, Reggie|      5|       18| 21635.94997024536|
|    15| Diebold, Debbie|     38|        3|216.80000114440918|
|    20|    Pulver, Eddy|      5|        7| 23702.40003967285|
|    21|       Kan, Adam|     39|        1|             105.0|
|    23|Bostwick, Roscoe|     13|        2|393.23999786376953|
+------+----------------+-------+---------+------------------+
only showing top 5 rows



### 3. Create a curated database
- And load the above created dataframes into tables

In [ ]:
%sql

DROP DATABASE IF EXISTS curated CASCADE;

CREATE DATABASE curated; 

In [ ]:
%sql

DROP TABLE IF EXISTS curated.completed_orders_metrics;
DROP TABLE IF EXISTS curated.refunded_orders_metrics;
DROP TABLE IF EXISTS curated.other_orders_metrics;
DROP TABLE IF EXISTS curated.completed_customer_orders;
DROP TABLE IF EXISTS curated.rfm_metrics;

In [ ]:
#Run the following commands to load the dataframes into tables in the curated database.

completed_orders_metrics.write.saveAsTable("curated.completed_orders_metrics")

refunded_orders_metrics.write.saveAsTable("curated.refunded_orders_metrics")

other_orders_metrics.write.saveAsTable("curated.other_orders_metrics")

completed_customer_orders.write.saveAsTable("curated.completed_customer_orders")

rfm_df.write.saveAsTable("curated.rfm_metrics")

#### Check the curated tables

In [ ]:
%sql

USE curated;
SHOW TABLES;

database,tableName,isTemporary
curated,completed_customer_orders,false
curated,completed_orders_metrics,false
curated,other_orders_metrics,false
curated,refunded_orders_metrics,false
curated,rfm_metrics,false


In [ ]:
%sql

SELECT * 
FROM curated.completed_orders_metrics 
limit 10;

Category,Year,Month_Year,Revenue,TotalOrders
Entertainment,2020,Dec-2020,5507960.012482643,3031
Appliances,2020,Dec-2020,5082387.872704029,6989
Superstore,2020,Dec-2020,90084.55527067184,660
School & Education,2020,Dec-2020,2807.4824171066284,51
Home & Living,2020,Dec-2020,186017.76805210114,1724
Women's Fashion,2020,Dec-2020,598978.1214361191,2623
Kids & Baby,2020,Dec-2020,57219.67628479004,640
Others,2020,Dec-2020,2388.3879946395755,47
Men's Fashion,2020,Dec-2020,714339.7039384842,5538
Computing,2020,Dec-2020,1252693.5991449356,1173


In [ ]:
%sql

SELECT * 
FROM curated.rfm_metrics
limit 10;

CustID,FullName,Recency,Frequency,Monetary
4,"Doughty, Reggie",5,18,21635.94997024536
15,"Diebold, Debbie",38,3,216.80000114440918
20,"Pulver, Eddy",5,7,23702.40003967285
21,"Kan, Adam",39,1,105.0
23,"Bostwick, Roscoe",13,2,393.23999786376953
28,"Drain, Reinaldo",55,1,70.0
32,"Horne, Reginald",7,97,47835.18991851807
33,"Rapoza, Darnell",11,49,32907.73117160797
41,"Batty, Angelo",50,1,219.89999389648438
44,"Ro, Kendall",21,3,4143.3790283203125
